In [6]:
import pandas as pd
import json
import requests
from datetime import datetime, timedelta
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from urllib.error import HTTPError
import certifi
import nbformat

## Connecting to Coinbase public API 

In [7]:
# Choose currency that should be used to display data
FIAT_CURRENCIES = ['EUR','USD']
MY_QUOTE_CURRENCY = FIAT_CURRENCIES[0]

# I am only interested in a few currencies that I want to trade, so let's add them here:
MY_CRYPTO_CURRENCIES = ["BTC","ETH","LTC"] 
GRANULARITIES = ['DAILY','60MIN','15MIN','1MIN']

CANDLE_INCREASE_COLOR = 'cyan'
CANDLE_DECREASE_COLOR = 'gray'
MACD_COLOR = '#A9B1F5'
MACD_SIGNAL_COLOR = '#FFD1A3'
RED_COLOR = '#D61170'
GREEN_COLOR = '#3FFA68'
#PLOTLY_TEMPLATE = 'plotly_dark'
PLOTLY_TEMPLATE = 'plotly'

# Connect to Coinbase

In [8]:
def connect(url, *args, **kwargs):
    try:
        if kwargs.get('param', None) is not None:
            params = kwargs.get('param')
            response = requests.get(url,params)
        else:
            response = requests.get(url)
        # print if an error occurs 
        response.raise_for_status()
        print(f'HTTP connection {url} successful!')
        return response
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')

In [9]:
server_time = json.loads(connect('https://api.exchange.coinbase.com/time').text)
# Server time does not comply to iso format, therefore slight modification of string needed
server_time_now = datetime.fromisoformat(server_time['iso'].replace('T', ' ', 1)[0:19])
print(f'Time on server: {server_time_now}')
print(f'Time on client: {datetime.now()}')

HTTP connection https://api.exchange.coinbase.com/time successful!
Time on server: 2023-09-09 12:03:33
Time on client: 2023-09-09 20:03:33.507642


## 2. Load statistic from last 24 hours 

In [10]:
currency_rows = []
for currency in MY_CRYPTO_CURRENCIES:
    data = json.loads(connect('https://api.exchange.coinbase.com/products/'+currency+'-'+MY_QUOTE_CURRENCY+'/stats').text)
    currency_rows.append(data)
df_24hstats = pd.DataFrame(currency_rows, index = MY_CRYPTO_CURRENCIES)
df_24hstats['base_currency'] = df_24hstats.index
df_24hstats['quote_currency'] = MY_QUOTE_CURRENCY
df_24hstats['open'] = df_24hstats['open'].astype(float)
df_24hstats['high'] = df_24hstats['high'].astype(float)
df_24hstats['low'] = df_24hstats['low'].astype(float)
df_24hstats['volume'] = df_24hstats['volume'].astype(float)
df_24hstats['last'] = df_24hstats['last'].astype(float)
df_24hstats['volume_30day'] = df_24hstats['volume_30day'].astype(float)
df_24hstats['performance'] = ((df_24hstats['last']-df_24hstats['open']) / df_24hstats['open']) * 100
df_24hstats_formatted = df_24hstats.copy()
df_24hstats_formatted['performance'] = df_24hstats_formatted['performance'].apply(lambda x: "{:.2f}%".format((x)))
df_24hstats_formatted['open'] = df_24hstats_formatted['open'].apply(lambda x: "{:,.2f}€".format((x)))
df_24hstats_formatted['high'] = df_24hstats_formatted['high'].apply(lambda x: "{:,.2f}€".format((x)))
df_24hstats_formatted['low'] = df_24hstats_formatted['low'].apply(lambda x: "{:,.2f}€".format((x)))
df_24hstats_formatted['last'] = df_24hstats_formatted['last'].apply(lambda x: "{:,.2f}€".format((x)))
print('Performance within last 24 hours in ' + MY_QUOTE_CURRENCY + ':')

HTTP connection https://api.exchange.coinbase.com/products/BTC-EUR/stats successful!
HTTP connection https://api.exchange.coinbase.com/products/ETH-EUR/stats successful!
HTTP connection https://api.exchange.coinbase.com/products/LTC-EUR/stats successful!
Performance within last 24 hours in EUR:


In [11]:
df_24hstats

,open,high,low,last,volume,volume_30day,base_currency,quote_currency,performance
BTC,24152.08,24240.93,24041.33,24175.25,232.536321,10654.693060,BTC,EUR,0.095934
ETH,1521.93,1531.46,1517.96,1527.92,1833.721390,90319.304336,ETH,EUR,0.393579
LTC,58.58,58.79,58.21,58.68,8963.377564,332390.213949,LTC,EUR,0.170707


In [22]:
# Convenience method that establishes a connection to Coinbase and will retrieve historic data for the chosen interval
def get_historic_data(start_date, end_date, interval, base_currency, quote_currency):

    VALID_INTERVAL = {'DAILY', '60MIN', '15MIN', '1MIN'}

    if interval not in VALID_INTERVAL:
        raise ValueError("results: interval must be one of %r." % VALID_INTERVAL)

    if interval == '1MIN':
        granularity = '60'
    elif interval == '15MIN':
        granularity = '900'
    elif interval == '60MIN':
        granularity = '3600'
    else: # DAILY as the default
        granularity = '86400'

    params = {'start':start_date, 'end':end_date, 'granularity':granularity}
    data = json.loads(connect('https://api.exchange.coinbase.com/products/'+quote_currency+'-'+base_currency+'/candles', param = params).text)
    [x.append(quote_currency) for x in data]
    [x.append(interval) for x in data]
    
    return data

### Step Three: Load Historic Data With Different Granularities

In [23]:
currency_history_rows = []
for currency in MY_CRYPTO_CURRENCIES:
    end_date = server_time_now.isoformat()
    # 1 minutes data:
    start_date = (server_time_now - timedelta(hours=2)).isoformat()
    currency_history_rows.extend(get_historic_data(start_date,end_date,'1MIN', MY_QUOTE_CURRENCY, currency))
    # 15 minutes data:
    start_date = (server_time_now - timedelta(hours=75)).isoformat()
    currency_history_rows.extend(get_historic_data(start_date,end_date,'15MIN', MY_QUOTE_CURRENCY, currency))
    # 60 minutes data:
    start_date = (server_time_now - timedelta(hours=300)).isoformat()
    currency_history_rows.extend(get_historic_data(start_date,end_date,'60MIN', MY_QUOTE_CURRENCY, currency))
    # Daily data:
    start_date = (server_time_now - timedelta(days=90)).isoformat()
    currency_history_rows.extend(get_historic_data(start_date,end_date,'DAILY', MY_QUOTE_CURRENCY, currency))
df_history = pd.DataFrame(currency_history_rows)
# Add column names in line with the Coinbase documentation
df_history.columns = ['time','low','high','open','close','volume','base_currency','granularity']
# We will add a few more columns just for better readability
df_history['quote_currency'] = MY_QUOTE_CURRENCY
df_history['date'] = pd.to_datetime(df_history['time'], unit='s')
df_history['year'] = pd.DatetimeIndex(df_history['date']).year
df_history['month'] = pd.DatetimeIndex(df_history['date']).month
df_history['day'] = pd.DatetimeIndex(df_history['date']).day
df_history['hour'] = pd.DatetimeIndex(df_history['date']).hour
df_history['minute'] = pd.DatetimeIndex(df_history['date']).minute

HTTP connection https://api.exchange.coinbase.com/products/BTC-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/BTC-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/BTC-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/BTC-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/ETH-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/ETH-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/ETH-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/ETH-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/LTC-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/LTC-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/LTC-EUR/candles successful!
HTTP connection https://api.exchange.coinba

In [24]:
print(f'Daily data included in df_history (in {MY_QUOTE_CURRENCY}):')
df_history.query('granularity == \'DAILY\'').head(5)

Daily data included in df_history (in EUR):


,time,low,high,open,close,volume,base_currency,granularity,quote_currency,date,year,month,day,hour,minute
719,1694217600,24123.31,24223.11,24219.11,24152.33,78.672047,BTC,DAILY,EUR,2023-09-09,2023,9,9,0,0
720,1694131200,23975.37,24691.50,24554.05,24218.24,412.350398,BTC,DAILY,EUR,2023-09-08,2023,9,8,0,0
721,1694044800,23943.30,24713.33,24019.89,24555.27,422.839696,BTC,DAILY,EUR,2023-09-07,2023,9,7,0,0
722,1693958400,23671.03,24269.09,24043.11,24020.42,322.194375,BTC,DAILY,EUR,2023-09-06,2023,9,6,0,0
723,1693872000,23690.00,24153.16,23916.97,24047.35,336.243297,BTC,DAILY,EUR,2023-09-05,2023,9,5,0,0


In [12]:
print(f'60 minute data included in df_history (in {MY_QUOTE_CURRENCY}):')
df_history.query('granularity == \'60MIN\'').tail(5)

60 minute data included in df_history (in EUR):


,time,low,high,open,close,volume,base_currency,granularity,quote_currency,date,year,month,day,hour,minute
2269,1693184400,59.60,60.12,59.97,60.01,291.558883,LTC,60MIN,EUR,2023-08-28 01:00:00,2023,8,28,1,0
2270,1693180800,59.77,60.57,60.56,59.96,302.187309,LTC,60MIN,EUR,2023-08-28 00:00:00,2023,8,28,0,0
2271,1693177200,60.37,60.58,60.44,60.58,174.629988,LTC,60MIN,EUR,2023-08-27 23:00:00,2023,8,27,23,0
2272,1693173600,60.23,60.45,60.35,60.45,150.962470,LTC,60MIN,EUR,2023-08-27 22:00:00,2023,8,27,22,0
2273,1693170000,60.26,60.62,60.39,60.35,836.391404,LTC,60MIN,EUR,2023-08-27 21:00:00,2023,8,27,21,0


#### Step 5 - Get More Insights

The enhanced data is stored in a pandas dataframe called df_history_enhanced. We will again use the four different granularities defined above.

In [13]:
AVERAGE_TRUE_RANGE_PERIODS = 14
currency_history_rows_enhanced = []
for currency in MY_CRYPTO_CURRENCIES:
    for granularity in GRANULARITIES:
        df_history_currency = df_history.query('granularity == @granularity & base_currency == @currency').copy()
        # Oldest to newest date sorting needed for SMA calculation
        df_history_currency = df_history_currency.sort_values(['date'], ascending=True)
        df_history_currency['SMA3'] = df_history_currency['close'].rolling(window=3).mean()
        df_history_currency['SMA7'] = df_history_currency['close'].rolling(window=7).mean()
        df_history_currency['EMA12'] = df_history_currency['close'].ewm(span=12, adjust=False).mean()
        df_history_currency['EMA26'] = df_history_currency['close'].ewm(span=26, adjust=False).mean()
        df_history_currency['MACD'] = df_history_currency['EMA12'] - df_history_currency['EMA26']
        df_history_currency['MACD_signal'] = df_history_currency['MACD'].ewm(span=9, adjust=False).mean()
        df_history_currency['macd_histogram'] = ((df_history_currency['MACD']-df_history_currency['MACD_signal']))
        df_history_currency['open_to_close_perf'] = ((df_history_currency['close']-df_history_currency['open']) / df_history_currency['open'])
        df_history_currency['high_low_diff'] = (df_history_currency['high']-df_history_currency['low'])
        df_history_currency['high_prev_close_abs'] = np.abs(df_history_currency['high'] - df_history_currency['close'].shift())
        df_history_currency['low_prev_close_abs'] = np.abs(df_history_currency['low'] - df_history_currency['close'].shift())
        df_history_currency['true_range'] = df_history_currency[['high_low_diff', 'high_prev_close_abs', 'low_prev_close_abs']].max(axis=1)
        #df_history_currency['average_true_range'] = df_history_currency['true_range'].rolling(window=14).sum()/14
        df_history_currency['average_true_range'] = df_history_currency['true_range'].ewm(alpha=1/AVERAGE_TRUE_RANGE_PERIODS, min_periods=AVERAGE_TRUE_RANGE_PERIODS, adjust=False).mean()
        df_history_currency['high_low_perf'] = ((df_history_currency['high']-df_history_currency['low']) / df_history_currency['high'])
        df_history_currency['open_perf_last_3_period_abs'] = df_history_currency['open'].rolling(window=4).apply(lambda x: x.iloc[1] - x.iloc[0])
        df_history_currency['open_perf_last_3_period_per'] = df_history_currency['open'].rolling(window=4).apply(lambda x: (x.iloc[1] - x.iloc[0])/x.iloc[0])
        df_history_currency['bull_bear'] = np.where(df_history_currency['macd_histogram']< 0, 'Bear', 'Bull')
        currency_history_rows_enhanced.append(df_history_currency)
df_history_enhanced = pd.concat(currency_history_rows_enhanced, ignore_index=True)
# Last step to tag changes in market trends from one period to the other (sorting important)
df_history_enhanced = df_history_enhanced.sort_values(['base_currency','granularity','date'], ascending=True)
df_history_enhanced['market_trend_continued'] = df_history_enhanced.bull_bear.eq(df_history_enhanced.bull_bear.shift()) & df_history_enhanced.base_currency.eq(df_history_enhanced.base_currency.shift()) & df_history_enhanced.granularity.eq(df_history_enhanced.granularity.shift())
df_history_enhanced.head(10)

,time,low,high,open,close,volume,base_currency,granularity,quote_currency,date,...,high_low_diff,high_prev_close_abs,low_prev_close_abs,true_range,average_true_range,high_low_perf,open_perf_last_3_period_abs,open_perf_last_3_period_per,bull_bear,market_trend_continued
390,1693977300,23968.42,23982.85,23970.70,23977.94,0.357180,BTC,15MIN,EUR,2023-09-06 05:15:00,...,14.43,NaN,NaN,14.43,NaN,0.000602,NaN,NaN,Bull,False
391,1693978200,23961.07,23977.50,23974.97,23964.87,0.712899,BTC,15MIN,EUR,2023-09-06 05:30:00,...,16.43,0.44,16.87,16.87,NaN,0.000685,NaN,NaN,Bear,False
392,1693979100,23960.68,23972.74,23967.00,23968.31,0.981368,BTC,15MIN,EUR,2023-09-06 05:45:00,...,12.06,7.87,4.19,12.06,NaN,0.000503,NaN,NaN,Bear,True
393,1693980000,23968.12,23993.38,23968.12,23984.43,1.822414,BTC,15MIN,EUR,2023-09-06 06:00:00,...,25.26,25.07,0.19,25.26,NaN,0.001053,4.27,0.000178,Bear,True
394,1693980900,23981.12,24005.63,23984.53,23983.34,1.188902,BTC,15MIN,EUR,2023-09-06 06:15:00,...,24.51,21.20,3.31,24.51,NaN,0.001021,-7.97,-0.000332,Bull,False
395,1693981800,23981.10,23994.87,23983.41,23988.62,1.195853,BTC,15MIN,EUR,2023-09-06 06:30:00,...,13.77,11.53,2.24,13.77,NaN,0.000574,1.12,0.000047,Bull,True
396,1693982700,23987.37,24012.58,23991.51,23998.35,1.647452,BTC,15MIN,EUR,2023-09-06 06:45:00,...,25.21,23.96,1.25,25.21,NaN,0.001050,16.41,0.000685,Bull,True
397,1693983600,23990.36,24005.13,23996.81,23994.27,3.166262,BTC,15MIN,EUR,2023-09-06 07:00:00,...,14.77,6.78,7.99,14.77,NaN,0.000615,-1.12,-0.000047,Bull,True
398,1693984500,23951.13,23997.93,23991.83,23996.29,8.593170,BTC,15MIN,EUR,2023-09-06 07:15:00,...,46.80,3.66,43.14,46.80,NaN,0.001950,8.10,0.000338,Bull,True
399,1693985400,23985.70,24001.62,23996.25,24001.62,2.482693,BTC,15MIN,EUR,2023-09-06 07:30:00,...,15.92,5.33,10.59,15.92,NaN,0.000663,5.30,0.000221,Bull,True


### Step 6 - Print Out Selected Information
Now, we can leverage the data which is available in df_history_enhanced to compile some basic parameters that we might want to use in a personal dashboard down the line.

In [14]:
additional_info_rows = []
for currency in MY_CRYPTO_CURRENCIES:
    df_history_enhanced = df_history_enhanced.sort_values(['date'], ascending=True) #oldest to newest
    row_data = [currency,
        float(df_24hstats.query('base_currency == @currency')['last'].to_string(index=False)), # current value
        df_history_enhanced.query('granularity == \'1MIN\' and base_currency == @currency').tail(1)['bull_bear'].to_string(index=False), # 1min market trend
        df_history_enhanced.query('granularity == \'1MIN\' and base_currency == @currency').iloc[-2]['bull_bear'], # prev 1min market trend
        df_history_enhanced.query('granularity == \'15MIN\' and base_currency == @currency').tail(1)['bull_bear'].to_string(index=False), #15min market trend
        df_history_enhanced.query('granularity == \'15MIN\' and base_currency == @currency').iloc[-2]['bull_bear'], # prev 15min market trend
        df_history_enhanced.query('granularity == \'60MIN\' and base_currency == @currency').tail(1)['bull_bear'].to_string(index=False), # 60min market trend
        df_history_enhanced.query('granularity == \'60MIN\' and base_currency == @currency').iloc[-2]['bull_bear'], # prev 60min market trend
        df_history_enhanced.query('granularity == \'60MIN\' and base_currency == @currency').tail(24)['low'].min(), # lowest last 24h
        df_history_enhanced.query('granularity == \'DAILY\' and base_currency == @currency').tail(30)['low'].min(), # lowest last 30d
        df_history_enhanced.query('granularity == \'DAILY\' and base_currency == @currency').tail(90)['low'].min(), # lowest last 90d
        df_history_enhanced.query('granularity == \'60MIN\' and base_currency == @currency').tail(24)['high'].max(), # highest last 24h
        df_history_enhanced.query('granularity == \'DAILY\' and base_currency == @currency').tail(30)['high'].max(), # highest last 30d
        df_history_enhanced.query('granularity == \'DAILY\' and base_currency == @currency').tail(90)['high'].max(), # highest last 90d
        round(df_history_enhanced.query('granularity == \'DAILY\' and base_currency == @currency').tail(90)['high_low_perf'].max(),3), # max daily high/low difference last 90d
        round(df_history_enhanced.query('granularity == \'60MIN\' and base_currency == @currency')['high_low_perf'].max(),3) # max daily high/low difference last 12.5d
        ]
    additional_info_rows.append(row_data)
df_additional_info = pd.DataFrame(additional_info_rows, columns = ['base_currency',
                                                                    'current_val',
                                                                    '1min_trend',
                                                                    '1min_prev_trend',
                                                                    '15min_trend',
                                                                    '15min_prev_trend',
                                                                    '60min_trend',
                                                                    '60min_prev_trend',
                                                                    '24h_low',
                                                                    '30day_low',
                                                                    '90day_low',
                                                                    '24h_high',
                                                                    '30day_high',
                                                                    '90day_high',
                                                                    '90day_max_high_low_span',
                                                                    '12day_max_high_low_span'])
df_additional_info['current_val'] = df_additional_info['current_val'].apply(lambda x: "{:,.2f}€".format((x)))
df_additional_info['24h_low'] = df_additional_info['24h_low'].apply(lambda x: "{:,.2f}€".format((x)))
df_additional_info['30day_low'] = df_additional_info['30day_low'].apply(lambda x: "{:,.2f}€".format((x)))
df_additional_info['90day_low'] = df_additional_info['90day_low'].apply(lambda x: "{:,.2f}€".format((x)))
df_additional_info['24h_high'] = df_additional_info['24h_high'].apply(lambda x: "{:,.2f}€".format((x)))
df_additional_info['30day_high'] = df_additional_info['30day_high'].apply(lambda x: "{:,.2f}€".format((x)))
df_additional_info['90day_high'] = df_additional_info['90day_high'].apply(lambda x: "{:,.2f}€".format((x)))
df_additional_info['90day_max_high_low_span'] = (df_additional_info['90day_max_high_low_span']*100).apply(lambda x: "{:.2f}%".format((x)))
df_additional_info['12day_max_high_low_span'] = (df_additional_info['12day_max_high_low_span']*100).apply(lambda x: "{:.2f}%".format((x)))
df_additional_info.set_index('base_currency').transpose()

base_currency,BTC,ETH,LTC
current_val,"24,195.26€","1,527.93€",58.61€
1min_trend,Bear,Bull,Bear
1min_prev_trend,Bull,Bull,Bear
15min_trend,Bull,Bull,Bear
15min_prev_trend,Bull,Bull,Bear
60min_trend,Bull,Bull,Bull
60min_prev_trend,Bull,Bull,Bull
24h_low,"23,975.37€","1,510.48€",57.49€
30day_low,"23,113.00€","1,412.04€",55.53€
90day_low,"22,782.25€","1,412.04€",55.53€


### Step 7 - Visualize Data
For visualization, we will use the powerful plotly library. It will give us the possibility to slice and dice data in an interactive way. Exactly what we need for an in-depth analysis of our daily market data.

Simple Visualization
First, let's create a simple OHLC (Open, High, Low, Close) candle stick chart for each of our currencies.

In [15]:
df_history_enhanced = df_history_enhanced.sort_values(['date'], ascending=True) #oldest to newest for visualization
granularity = '60MIN'
for currency in MY_CRYPTO_CURRENCIES:
    df_history_for_chart = df_history_enhanced.query(f'granularity == \'{granularity}\' and base_currency == \'{currency}\'')
    fig = go.Figure()
    fig.add_trace(go.Candlestick(
        x=df_history_for_chart['date'],
        open=df_history_for_chart['open'],
        high=df_history_for_chart['high'],
        low=df_history_for_chart['low'],
        close=df_history_for_chart['close'],
        name='OHLC'))
    fig.add_trace(go.Scatter(
        x=df_history_for_chart['date'],
        y=df_history_for_chart['average_true_range'],
        line=dict(color=MACD_COLOR),
        mode='lines',
        name="ATR",
        yaxis="y2"
    ))
    cs = fig.data[0]
    cs.increasing.fillcolor = CANDLE_INCREASE_COLOR
    cs.increasing.line.color = CANDLE_INCREASE_COLOR
    cs.decreasing.fillcolor = CANDLE_DECREASE_COLOR
    cs.decreasing.line.color = CANDLE_DECREASE_COLOR
    fig.update_layout(
        yaxis=dict(
            title=MY_QUOTE_CURRENCY,
            side="right",
        ),
        yaxis2=dict(
            title="Averate True Range (Volatility)",
            anchor="free",
            overlaying="y",
            side="left",
        ),
        yaxis2_showgrid = False,
        yaxis2_zeroline = False,
        xaxis_rangeslider_visible=False,
        title=f'OHLC Chart for {currency} with time interval {granularity}',
        hovermode='x unified',
        xaxis=dict(
            rangeslider=dict(visible=True),
            type="date"
        ),
        template = PLOTLY_TEMPLATE,)
    fig.show()

/Users/richy_air/Documents/project/GPT_Crypto_Analysis/.venv/lib/python3.9/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



### Advanced Visualization
Now that we have drawn a basic OHLC chart, let's get a bit more fancy. We will now draw the chart again. But this time we will add the following information:

MACD line
MACD signal line
Colored area to show if we are currently in a bull market (green) or bear market (red) in each given period
Before we do that, there is one more important step to take: we must construct the data needed to color different periods for the market trends (bull versus bear). Therefore, let's create another pandas dataframe df_history_market_trend_intervals.

In [18]:
market_trend_interval_rows = []
for currency in MY_CRYPTO_CURRENCIES:
    for granularity in GRANULARITIES:
        df_history_market_trend_intervals = df_history_enhanced.query('base_currency == @currency and market_trend_continued == False and granularity == @granularity').copy()
        #df_history_market_trend_intervals['previous_period_date'] = df_history_market_trend_intervals.date.shift()
        df_history_market_trend_intervals['next_period_date'] = df_history_market_trend_intervals.date.shift(-1)
        df_history_market_trend_intervals['next_period_date'] = df_history_market_trend_intervals['next_period_date']
        df_history_market_trend_intervals.next_period_date = df_history_market_trend_intervals.next_period_date.fillna(datetime.now())
        df_history_market_trend_intervals['color'] = df_history_market_trend_intervals['bull_bear'].apply(lambda x: GREEN_COLOR if x == 'Bull' else RED_COLOR)
        df_history_market_trend_intervals = df_history_market_trend_intervals[['base_currency','granularity','bull_bear','color','date','next_period_date']].rename(columns={"date": "start_date", "next_period_date": "finish_date"})
        market_trend_interval_rows.append(df_history_market_trend_intervals)
df_history_market_trend_intervals = pd.concat(market_trend_interval_rows, ignore_index=True)

In [19]:
granularity = '60MIN'
for currency in MY_CRYPTO_CURRENCIES:
    df_history_for_chart = df_history_enhanced.query(f'granularity == \'{granularity}\' and base_currency == \'{currency}\'').copy()
    df_history_market_trend_for_chart = df_history_market_trend_intervals.query(f'granularity == \'{granularity}\' and base_currency == \'{currency}\'').copy()
    macd_min = df_history_for_chart['MACD'].min()
    macd_max = df_history_for_chart['MACD'].max()
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Ohlc(
        x=df_history_for_chart['date'],
        open=df_history_for_chart['open'],
        high=df_history_for_chart['high'],
        low=df_history_for_chart['low'],
        close=df_history_for_chart['close'],
        increasing_line_color= CANDLE_INCREASE_COLOR,
        decreasing_line_color= CANDLE_DECREASE_COLOR,
        name='OHLC'),
        secondary_y=True),
    fig.add_trace(go.Scatter(
        x=df_history_for_chart['date'],
        y=df_history_for_chart['MACD'],
        name='MACD',
        line=dict(color=MACD_COLOR),
        mode='lines'),
        secondary_y=False)
    fig.add_trace(go.Scatter(
        x=df_history_for_chart['date'],
        y=df_history_for_chart['MACD_signal'],
        name='MACD Signal',
        line=dict(color=MACD_SIGNAL_COLOR),
        mode='lines'),
        secondary_y=False)
    for i in range(df_history_market_trend_for_chart['base_currency'].count()):
        fig.add_vrect(x0=df_history_market_trend_for_chart.iloc[i]['start_date'],
            x1=df_history_market_trend_for_chart.iloc[i]['finish_date'],
            col=1,
            #fillcolor=df_history_market_trend_for_chart.iloc[i]['color'],
            line=dict(color=df_history_market_trend_for_chart.iloc[i]['color']),
            opacity=1.0,
            line_width=1)
    fig.update_layout(
        title=f'OHLC Chart for {currency} with time interval {granularity}',
        hovermode='x unified',
        legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.94),
        #autosize=False,
        #width=900,
        #height=600,
        template = PLOTLY_TEMPLATE,
        yaxis_range=[macd_min,macd_max],
        yaxis1_showgrid = False,
        yaxis1_showticklabels = False,
        yaxis1_zeroline = False,
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        ),
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(visible=True),
            type="date"
        )
    )
    if granularity == 'DAILY':
        fig.update_layout(xaxis_tickformat = '%d %b (%a)')
    else:
        fig.update_layout(xaxis_tickformat = '%d %b %H:%M')
    fig.show()

/Users/richy_air/Documents/project/GPT_Crypto_Analysis/.venv/lib/python3.9/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

